In [6]:
import scipy.integrate
import numpy as np
import matplotlib.pyplot as plt

# Update shooting guess: Algorithmic approach

We model an arbritary system on the domain $x\in [0,1]$.


$\frac{d^2y}{dx^2}=4(y-x)$

The following boundary conditions are given.

$y(0)=0,\;y(1)=2$

To solve the BVP with the shooting method we convert the ODE (2nd order) to the canonical form (1st order) using variable substitution.

$y_1\equiv y,\;y_2\equiv \frac{dy}{dx}$

The resulting ODE system:

$\frac{dy_1}{dx}=y_2$

$\frac{dy_2}{dx}=4(y_1-x)$

The new boundary conditions:

$y_1(0)=0,\;y_1(1)=2$

Define canonical ODE system.

In [7]:
def dy(x: np.ndarray, y: np.ndarray) -> np.ndarray:
    """Examplary ordinary differential equation.

    Parameters
    ----------
    x : np.ndarray
        Independent variable
    y : np.ndarray
        Dependent variables

    Returns
    -------
    np.ndarray
        First order derivative of dependent variables
    """
    y1, y2 = y
    dy1dx = y2
    dy2dx = 4*(y1-x)
    return np.array([dy1dx, dy2dx])

Define parameters.

In [8]:
# domain
x = np.linspace(0, 1, 101)

# known boundary conditions
y1_0 = 0
y1_f = 2

# guess initial condition, [gamma0, gamma1]
guess = [1, 0]

Find the optimal guess using the secant method.

In [9]:
# solve IVP for two initial guesses
sol0 = scipy.integrate.solve_ivp(dy, (x[0], x[-1]), [y1_0, guess[0]], t_eval=x)
sol1 = scipy.integrate.solve_ivp(dy, (x[0], x[-1]), [y1_0, guess[1]], t_eval=x)
y = [sol0.y[0], sol1.y[0]]  # save the solutions for y = y1 for each guess

max_iter = 100  # maximum number of iterations for secant method
tolerance = 1e-3  # error tolerance for stopping criteria

for i in range(max_iter):
    if abs(y[-1][-1]-y1_f) < tolerance:
        break  # if error tolerance is reached, stop the algorithm
    # approximate new guess with secant
    new_guess = guess[-1]-(y[-1][-1]-y1_f)*(guess[-1]-guess[-2])/(y[-1][-1]-y[-2][-1])
    guess.append(new_guess)
    # solve IVP using the new guess
    new_sol = scipy.integrate.solve_ivp(dy, (x[0], x[-1]), [y1_0, guess[-1]], t_eval=x)
    y.append(new_sol.y[0])

Plot the results.

In [ ]:
fig, ax = plt.subplots()
for i, y_i in enumerate(y):
    ax.plot(x, y_i, label=f"$y_{i}$")
ax.plot(1, 2, label="boundary condition $y_1(1)=2$", linestyle="", marker="x", color="m")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("BVP - shooting method")
ax.legend()
ax.grid()
fig.show()